In [1]:
__author__ = 'Mike Fitzpatrick <mike.fitzpatrick@noirlab.edu>, Robert Nikutta <robert.nikutta@noirlab.edu>'
__version__ = '20240603' #yyyymmdd
__datasets__ = []
__keywords__ = []

# How to use the Data Lab *Auth Client* Service
*Mike Fitzpatrick, Robert Nikutta, and the Astro Data Lab Team*

# Goals
To demonstrate the use of the Data Lab ``authClient`` API (login, logout, token validity checks, etc.)

# Summary
This notebook documents how to use the Data Lab authorization system via the *Auth Client* service. This can be done either from a Python script or from the command line using the <i>datalab</i> command.

While most Data Lab services may be accessed anonymously, others require a valid user account in order to take advantage of resources such as <i>Virtual Storage</i> or <i>Compute Services</i>.  User accounts may be created on the main Data Lab website and are not discussed here.

Assuming an account has been created, the <i><b>authClient</b></i> interface can be used to authenticate yourself to the Data Lab to obtain an identity token used to access the resources allocated to the account.  This is typically the first step in any notebook or application, the returned token is then passed to every other interface method that requires authenticated access to a resource or service.

A complete summary of the *AuthClient* interface is given below.

# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.

# Imports

Most notebooks will begin by importing whatever packages are required. In this case we'll want to import the ``authClient`` interface from the ``dl`` (i.e. Data Lab) package.

In [2]:
from getpass import getpass
from dl import authClient as ac, queryClient as qc

The next step is to call the ``login()`` method with your account name and password to obtain an identity token:

In [3]:
# Login using your user name and password, validate the returned
# token or else print the returned error message.
token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()
if not ac.isValidToken(token):
    print('Error: invalid login for user %s (%s)' % (username,token))
else:
    print("Login token:   %s" % token)

Enter user name: (+ENTER)  demo00
Enter password: (+ENTER)  ·······


Login token:   demo00.1018.1018.$1$LXsNrBh.$kSlN0QCwdRipDVVmczno60


The `login()` method will return either a valid token or an error message.  By calling the `isValidToken()` method it is possible to trap any login errors in the task.  The use of the ``getpass()`` function means passwords do not need to be exposed and notebooks can be shared more freely.

<font color='red'><h4>Security Note</h4></font>

<font color='green'>
Once you have logged in successfully, an identity token is saved on the server running the application (the Jupyter notebook server in this case) and will be offered for subsequent logins when no password is provided.  This means that notebooks may be shared to use a valid account without sharing the password in the notebook.  You can use the `logout()` method (see below) to remove this token from the server manually, but as an added precaution, tokens will not be used on public notebook servers, and the password will <b>always</b> be required when attempting to login as some user other than the logged-in user on authenticated notebook servers.
</font>

In [4]:
# An example of using the stored token to login to the Data Lab
tok2 = ac.login('anonymous')
print('New login token: %s' % tok2)
print('Is new token valid?  %s' % ac.isValidToken(tok2))

New login token: anonymous.0.0.anon_access
Is new token valid?  True


#### Additional Interface Methods

Users can check whether specific account information is correct:

In [5]:
print('Is "demo00" a valid user?  %s' % ac.isValidUser('demo00'))
print('Is "foobar" the current password?  %s' % ac.isValidPassword('demo00','foobar'))
print('Do we have a valid login token?  %s' % ac.isValidToken(token))

Is "demo00" a valid user?  True
Is "foobar" the current password?  False
Do we have a valid login token?  True


Users can also check on their login status using either the account name or the current token:

In [6]:
print('Is user "dldemo" currently logged-in?  %s' % ac.isUserLoggedIn ('demo00'))
print('Is my token currently logged-in:  %s' % ac.isTokenLoggedIn (token))
print('Is user "Scooby" currently logged-in?  %s' % ac.isUserLoggedIn ('Scooby'))

Is user "dldemo" currently logged-in?  True
Is my token currently logged-in:  True
Is user "Scooby" currently logged-in?  False


#### Logging <i>Out</i> of the Data Lab

The `logout()` method is used to log the user out of the Data Lab and remove the identity token from the server.  Strictly speaking, it is not required that an application log out once it is complete, subsequent logins will return the same identity token the next time you log in so long as it is still valid (e.g. you haven't logged out elsewhere)

In [7]:
# Log out the current user and remove the identity token.
print ('Is token logged in: ', ac.isTokenLoggedIn(token))
res = ac.logout(token)
print('Log out result = "%s"'% res)
print('Is token logged in following logout(): ', ac.isTokenLoggedIn(token))

Is token logged in:  True
Log out result = "OK"
Is token logged in following logout():  False


### Example:  Anonymous Data Query

All public data services may be accessed anonymously, when queried in this way data will always be returned immediately to the client.  When saving results to a Data Lab resource such as virtual storage, however, a valid user login token is required.

In [8]:
#anon_token = ac.login('anonymous')
#print(anon_token)
query = "select * from usno.a2 limit 3"
#response = qc.query(anon_token, adql=query, fmt='csv', output='foo.csv')
response = qc.query(sql=query)
print(response)

id,raj2000,dej2000,htm9,nest256,ring256,bmag,rmag,epoch,raj2000_,dej2000_,actflag,mflag
1200-16738677,314.9983,35.267575,3407871,212220,167296,16.7,15.1,1951.517,20:59:59.592,+35:16:03.27, , 
1200-16737889,314.994803,35.267942,3407871,212220,167296,17.2,16.3,1951.517,20:59:58.753,+35:16:04.59, , 
1200-16740156,315.005123,35.270559,3407871,212220,167296,19.7,18.8,1951.517,21:00:01.230,+35:16:14.01, , 



### Example:  Saving Results to Virtual Storage

Details of using the Virtual Storage system are described elsewhere, but in general the term *Virtual Storage* is used to refer to either a user's private database (their *MyDB*), or a remote file storage service that resides close to the data services.  *Virtual Storage* is used to allow workflows to be created that minimize the transfer of data.  

#### Saving to the remote file storage

Saving results to a file can be done by specifying the 'out' parameter in the form:

`vos://<filename>`

The `<filename>` must be unique for each query or else an error will be issued and the results will not be saved.  Therefore, good coding style would trap potential errors and handle it accordingly, e.g.

In [9]:
query = "select * from usno.b1 limit 20"
try:
    response = qc.query(token, adql=query, fmt='csv', out='vos://mags.csv')
except Exception as e:
    # Handle any errors in the query.  By running this cell multiple times with the same
    # output file, or by using a bogus SQL statement, you can view various error messages.
    print(e)
else:
    if response is not None: 
        print(response)           # print the response
    else:
        print("OK")

OK


#### Saving to the user's remote database

Similarly, results may be saved to a user's MyDB database by specifying a table name of the form:

`mydb://<tablename>`

As with filenames, the `<tablename>` must be unique.

In [10]:
# Drop the test table if it already exists
qc.mydb_drop('usno_test',token=token)

query = "select * from usno.b1 limit 1000"
try:
    response = qc.query (token, adql=query, fmt='csv', out='mydb://test1')
except Exception as e:
    # Handle any errors in the query.  By running this cell multiple times with the same
    # output file, or by using a bogus SQL statement, you can view various error messages.
    print(e)
else:
    if response is not None: 
        print(response)           # print the response
    else:
        print("OK")

OK


***

### Auth Client API Summary

A summary of the complete *AuthClient* API is shown below:

```
LOGIN METHODS:
--------------
             token = login  (user, password=None, debug=False, verbose=False)
               res =logout  (token)

VALIDATION METHODS:
-------------------
            bval = isAlive  (svc_url=DEF_SERVICE_URL)
       bval = isValidToken  (token)
        bval = isValidUser  (user)
    bval = isValidPassword  (user, password)

ACCESS METHODS:
---------------
          bval = hasAccess  (user, resource)
     bval = isUserLoggedIn  (user)
    bval = isTokenLoggedIn  (token)

ACCOUNT ADMIN METHODS:
----------------------
 new_token = passwordReset  (token, username, password)

         url = set_service  (svc_url)
               get_service  ()

               set_profile  (profile)
profile_name = get_profile  ()
 prof_list = list_profiles  (token)

To instantiate a Client object:

    authClient = getClient  ()
```